Its my friends birthday tomorrow and it regular custom that we write codes to wish each other happy birthda, so i swept my phone for all the birthday test messages i have sent people and i also checked the internet too. So i am going to build a RNN to generate him a birthday message.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


!pip install -q -U tensorflow>=2
import tensorflow as tf



try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import os
import time


ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.1 which is incompatible.
TensorFlow is already loaded. Please restart the runtime to change versions.


Read in the text

In [0]:
corpus = open("birthday_messages.txt", "rb").read().decode(encoding='utf-8')

In [3]:
print ('Length of text: {} characters'.format(len(corpus)))

Length of text: 226256 characters


In [4]:
print(corpus[:250])

Happy birthday to the best sister in d world, I know you know that's a lie. I pray that you grow in d knowledgeable of Gods word and continue to prosper. I love you, I know you know dats another lie


--------------------------

Happy birthday to you


In [5]:
# The unique characters in the file
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

83 unique characters


Vectorize text- map all unique characters to an integer

In [0]:
# Creating a mapping from unique characters to indices
import numpy as np
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in corpus])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  '*' :   7,
  ',' :   8,
  '-' :   9,
  '.' :  10,
  '/' :  11,
  '0' :  12,
  '1' :  13,
  '2' :  14,
  '3' :  15,
  '5' :  16,
  '6' :  17,
  '9' :  18,
  ':' :  19,
  ...
}


In [8]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(corpus[:25]), text_as_int[:25]))

'Happy birthday to the bes' ---- characters mapped to int ---- > [29 48 63 63 72  1 49 56 65 67 55 51 48 72  1 67 62  1 67 55 52  1 49 52
 66]


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 50
examples_per_epoch = len(corpus)//(seq_length+1)

In [10]:
#convert stream of text into character indices
# Create training examples / targets

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

H
a
p
p
y


Divide characters into batch size of specific lenght

In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Happy birthday to the best sister in d world, I kno'
"w you know that's a lie. I pray that you grow in d "
'knowledgeable of Gods word and continue to prosper.'
' I love you, I know you know dats another lie\n\n\n---'
'-----------------------\n\nHappy birthday to you frie'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [13]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Happy birthday to the best sister in d world, I kn'
Target data: 'appy birthday to the best sister in d world, I kno'


The input of the RNN will start from '<' and predict '!' in the next time step till the final predicted output is 't' the last character in the sequence before it moves to the next sequence.

In [14]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 29 ('H')
  expected output: 48 ('a')
Step    1
  input: 48 ('a')
  expected output: 63 ('p')
Step    2
  input: 63 ('p')
  expected output: 63 ('p')
Step    3
  input: 63 ('p')
  expected output: 72 ('y')
Step    4
  input: 72 ('y')
  expected output: 1 (' ')


In [15]:
# Batch size
BATCH_SIZE = 82

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 1000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((82, 50), (82, 50)), types: (tf.int64, tf.int64)>

In [0]:
#Batch size is usually the size of your vocabulary because you will want to maintain a reasonable proportional embedding distance between your vocab



# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [19]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(82, 50, 83) # (batch_size, sequence_length, vocab_size)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (82, None, 256)           21248     
_________________________________________________________________
gru (GRU)                    (82, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (82, None, 83)            85075     
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.


In [21]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([51, 17, 80, 39,  2, 15, 37, 70, 12, 10, 26, 27, 41, 54,  5, 72, 30,
       50, 44, 57, 69, 14, 20, 67, 23, 20,  5, 65, 33, 69, 59, 74,  1, 48,
       40, 23, 43, 51, 51, 23, 61, 62, 48, 13, 74, 26,  9, 62, 20, 52])

Checking the sample text generated by this untrained model

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '--------------------------------------------------'

Next Char Predictions: 
 'd6“R!3Pw0.EFTg(yIcWjv2;tB;(rLvl{ aSBVddBnoa1{E-o;e'


Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [23]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (82, 50, 83)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.4240646


In [0]:
model.compile(optimizer='adam', loss=loss)

Configure checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
54/54 [==============================] - 169s 3s/step - loss: 3.6424
Epoch 2/20
54/54 [==============================] - 166s 3s/step - loss: 2.4401
Epoch 3/20
54/54 [==============================] - 164s 3s/step - loss: 2.0410
Epoch 4/20
54/54 [==============================] - 163s 3s/step - loss: 1.8702
Epoch 5/20
54/54 [==============================] - 165s 3s/step - loss: 1.7523
Epoch 6/20
54/54 [==============================] - 165s 3s/step - loss: 1.6392
Epoch 7/20
54/54 [==============================] - 166s 3s/step - loss: 1.5399
Epoch 8/20
54/54 [==============================] - 166s 3s/step - loss: 1.4576
Epoch 9/20
54/54 [==============================] - 165s 3s/step - loss: 1.3777
Epoch 10/20
54/54 [==============================] - 165s 3s/step - loss: 1.3069
Epoch 11/20
54/54 [==============================] - 164s 3s/step - loss: 1.2414
Epoch 12/20
54/54 [==============================] - 166s 3s/step - loss: 1.1763
Epoch 13/20
54/54 [==================

In [27]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            21248     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 83)             85075     
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 0.2

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [31]:
print(generate_text(model, start_string=u"Happy birthday "))

Happy birthday to my best friend that you simply have got a shock in the foremost of your distinctive day of yours be some person I hope you have got a good day.
- - - - - - -
“I hope you have got a good day.

- - - - - - -
“I hope you have got a good day.
- - - - - - -
“I hope you have got a good day. 

- 

"I trust you have got a good day.

- - - - - - -
“Happy Birthday to my best pal! I trust you have got a good day.

- - - - - - -
“I hope you have got a good day. Hope your birthday is brimful with fun and laughter. Have a beautiful birthday to my best pal! I trust you have got a good day.
- - - - - - -
“I hope you have got a good day.

- - - - - - -
“I hope you have got a good day.

-

"I trust you have got a good day.

- - - - - - -
“I hope you have got a good day. Hope your birthday is as astounding one for a prosperous years and see an opportune to recollect you have got a good day. 

- - - - - - - - -
“I hope you have got a good day.

- - - - - - -
“I hope you have got a good d